In [ ]:
%pip install snowflake-connector-python

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from datetime import datetime
import os

# Leer variables de entorno
SNOWFLAKE_ACCOUNT = os.getenv('SNOWFLAKE_ACCOUNT')
SNOWFLAKE_USER = os.getenv('SNOWFLAKE_USER')
SNOWFLAKE_PASSWORD = os.getenv('SNOWFLAKE_PASSWORD')
SNOWFLAKE_DATABASE = os.getenv('SNOWFLAKE_DATABASE')
SNOWFLAKE_SCHEMA_ANALYTICS = os.getenv('SNOWFLAKE_SCHEMA_ANALYTICS', 'ANALYTICS')
SNOWFLAKE_WAREHOUSE = os.getenv('SNOWFLAKE_WAREHOUSE', 'COMPUTE_WH')

In [ ]:
# Instalación de dependencias y descarga de JARs para Spark-Snowflake
!pip install snowflake-snowpark-python snowflake-connector-python

# Crear directorio para JARs si no existe
import os
jars_dir = '/content/jars'
os.makedirs(jars_dir, exist_ok=True)

# Descargar JARs necesarios para Spark 3.x con Scala 2.12
# Snowflake Spark Connector compatible con Spark 3.x y Scala 2.12
snowflake_jar_url = "https://repo1.maven.org/maven2/net/snowflake/spark-snowflake_2.12/2.12.0-spark_3.4/spark-snowflake_2.12-2.12.0-spark_3.4.jar"
snowflake_jdbc_url = "https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.14.4/snowflake-jdbc-3.14.4.jar"

# Descargar JARs
!wget -O {jars_dir}/spark-snowflake_2.12-2.12.0-spark_3.4.jar {snowflake_jar_url}
!wget -O {jars_dir}/snowflake-jdbc-3.14.4.jar {snowflake_jdbc_url}

print("JARs descargados exitosamente")

In [ ]:
!mkdir -p /content/jars
!wget -q -O /content/jars/snowflake-jdbc-3.14.4.jar https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.14.4/snowflake-jdbc-3.14.4.jar
!wget -q -O /content/jars/spark-snowflake_2.12-2.12.0-spark_3.4.jar https://repo1.maven.org/maven2/net/snowflake/spark-snowflake_2.12/2.12.0-spark_3.4/spark-snowflake_2.12-2.12.0-spark_3.4.jar
!ls -lh /content/jars


In [ ]:
# Importar PySpark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import random
from pyspark.sql import types as T

from datetime import datetime, timedelta


# Configuración de JARs para Snowflake
jars_dir = '/content/jars'
spark_jars = f"{jars_dir}/spark-snowflake_2.12-2.12.0-spark_3.4.jar,{jars_dir}/snowflake-jdbc-3.14.4.jar"

# Crear sesión Spark simple
print("Creando sesión Spark...")

spark = SparkSession.builder \
    .appName("NYC_Taxi_OBT_Analysis") \
    .config("spark.jars", "/content/jars/snowflake-jdbc-3.14.4.jar,/content/jars/spark-snowflake_2.12-2.12.0-spark_3.4.jar") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print(f" Spark iniciado correctamente. Versión: {spark.version}")


test_count = spark.range(5).count()
print(f"Spark funcionando correctamente - Test: {test_count} registros")
print(f"Versión Spark: {spark.version}")

# Mostrar configuración activa
print(f"JARs configurados: {len(spark_jars.split(','))} archivos")

In [ ]:
sfOptions = {
    "sfURL": SNOWFLAKE_ACCOUNT,
    "sfUser": SNOWFLAKE_USER,
    "sfPassword": SNOWFLAKE_PASSWORD,
    "sfDatabase": SNOWFLAKE_DATABASE,
    "sfSchema": SNOWFLAKE_SCHEMA_ANALYTICS,
    "sfWarehouse": SNOWFLAKE_WAREHOUSE,
}

df = (
    spark.read
        .format("net.snowflake.spark.snowflake")
        .options(**sfOptions)
        .option("dbtable", "OBT_TRIPS")
        .load()
)

df.show(5, truncate=False)


In [ ]:
# CONFIGURACIÓN INICIAL PARA VALIDACIONES
print("=== VALIDACIONES Y EXPLORACIÓN - OBT NYC TAXI ===")

# Conexión a Snowflake (reutilizar configuración del notebook anterior)
try:
    import snowflake.connector
    import time
    from datetime import datetime

    print("snowflake-connector-python disponible")

    # Configuración de conexión

    conn_config = {
        'user': SNOWFLAKE_USER,
        'password': SNOWFLAKE_PASSWORD,
        'account': SNOWFLAKE_ACCOUNT,
        'warehouse': SNOWFLAKE_WAREHOUSE,
        'database': SNOWFLAKE_DATABASE,
        'schema': SNOWFLAKE_SCHEMA_ANALYTICS
    }
    
    def execute_snowflake_sql(sql_query, fetch_results=True):
        """
        Ejecuta SQL directamente en Snowflake con resultados
        """
        try:
            conn = snowflake.connector.connect(**conn_config)
            cursor = conn.cursor()

            cursor.execute(sql_query)

            if fetch_results:
                results = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]
                return results, columns
            else:
                rows_affected = cursor.rowcount
                print(f" SQL ejecutado. Filas afectadas: {rows_affected:,}")
                return rows_affected, []

        except Exception as e:
            print(f"Error ejecutando SQL: {e}")
            return None, []
        finally:
            try:
                cursor.close()
                conn.close()
            except:
                pass

except ImportError:
    print("snowflake-connector-python no instalado")
    print("Instalar con: pip install snowflake-connector-python")

    def execute_snowflake_sql(sql_query, fetch_results=True):
        print("Connector no disponible. SQL para ejecutar manualmente:")
        print("-" * 60)
        print(sql_query)
        print("-" * 60)
        return None, []

print(" Configuración lista para preguntas de negocio en analytics.obt_trips")

## Debido al tiempo que tomaban en ejecutarse no se pudo ejecutar todo con spark, sin embargo se añadieron las consultas utilizando el conector de snowflake con python.

In [ ]:
# A. TOP 10 ZONAS PICKUP POR VOLUMEN MENSUAL
print("ANÁLISIS A: Top 10 Zonas Pickup por Volumen Mensual")

# Top 10 zonas de pickup por volumen mensual
top_pickup_monthly = (
    df.groupBy("month", "pu_zone", "pu_borough")
    .agg(F.count("*").alias("num_trips"))
    .orderBy("month", F.col("num_trips").desc())
)

print("TOP 10 ZONAS PICKUP POR VOLUMEN MENSUAL:")
top_pickup_monthly.show(10)

In [ ]:
# A. TOP 10 ZONAS PICKUP POR VOLUMEN MENSUAL - VERSIÓN SNOWFLAKE CONNECTOR
print("ANÁLISIS A: Top 10 Zonas Pickup por Volumen Mensual")

# Consulta SQL directa usando conector de Snowflake
sql_pickup_zones = """
SELECT
    month,
    pu_zone,
    pu_borough,
    COUNT(*) as num_trips
FROM analytics.obt_trips
WHERE pu_zone IS NOT NULL
GROUP BY month, pu_zone, pu_borough
ORDER BY month, COUNT(*) DESC
LIMIT 10
"""
execute_snowflake_sql(sql_pickup_zones)

In [ ]:
# B. TOP 10 ZONAS DROPOFF POR VOLUMEN MENSUAL
print("ANÁLISIS B: Top 10 Zonas Dropoff por Volumen Mensual")

# Top 10 zonas de dropoff por volumen mensual
top_dropoff_monthly = (
    df.groupBy("month", "do_zone", "do_borough")
    .agg(F.count("*").alias("num_trips"))
    .orderBy("month", F.col("num_trips").desc())
)

print("TOP 10 ZONAS DROPOFF POR VOLUMEN MENSUAL:")
top_dropoff_monthly.show(10)

In [ ]:
# B. TOP 10 ZONAS DROPOFF POR VOLUMEN MENSUAL
print("ANÁLISIS B: Top 10 Zonas Dropoff por Volumen Mensual")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    month,
    do_zone,
    do_borough,
    COUNT(*) as num_trips
FROM analytics.obt_trips
WHERE do_zone IS NOT NULL
GROUP BY month, do_zone, do_borough
ORDER BY month ASC, COUNT(*) DESC
limit 10
"""
execute_snowflake_sql(query)

In [ ]:
# C. EVOLUCIÓN MENSUAL DE TOTAL_AMOUNT Y TIP_PCT POR BOROUGH
print("ANÁLISIS C: Evolución Mensual de Total_Amount y Tip_Pct por Borough")

# Evolución mensual de total_amount y tip_pct por borough
evolution_monthly = (
    df.groupBy("year", "month", "pu_borough")
    .agg(F.round(F.avg("total_amount"), 2).alias("avg_total_amount"),
         F.round(F.avg("tip_pct"), 2).alias("avg_tip_pct"))
    .orderBy("year", "month", "pu_borough")
)

print("EVOLUCIÓN MENSUAL DE TOTAL_AMOUNT Y TIP_PCT POR BOROUGH:")
evolution_monthly.show(10)

In [ ]:
# C. EVOLUCIÓN MENSUAL DE TOTAL_AMOUNT Y TIP_PCT POR BOROUGH
print("ANÁLISIS C: Evolución Mensual de Total_Amount y Tip_Pct por Borough")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    year,
    month,
    pu_borough,
    ROUND(AVG(total_amount), 2) AS avg_total_amount,
    ROUND(AVG(tip_pct), 2) AS avg_tip_pct
FROM analytics.obt_trips
WHERE pu_borough IS NOT NULL
GROUP BY year, month, pu_borough
ORDER BY year, month, pu_borough;
"""
execute_snowflake_sql(query)

In [ ]:
# D. TICKET PROMEDIO POR SERVICE_TYPE Y MES
print("ANÁLISIS D: Ticket Promedio por Service_Type y Mes")

# Ticket promedio por service_type y mes
ticket_monthly = (
    df.groupBy("year", "month", "service_type")
    .agg(F.round(F.avg("total_amount"), 2).alias("avg_ticket"))
    .orderBy("year", "month", "service_type")
)

print("TICKET PROMEDIO POR SERVICE_TYPE Y MES:")
ticket_monthly.show(50)

In [ ]:
# D. TICKET PROMEDIO POR SERVICE_TYPE Y MES
print("ANÁLISIS D: Ticket Promedio por Service_Type y Mes")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    year,
    month,
    service_type,
    ROUND(AVG(total_amount), 2) AS avg_ticket
FROM analytics.obt_trips
WHERE service_type IS NOT NULL
    AND total_amount IS NOT NULL
GROUP BY year, month, service_type
ORDER BY year, month, service_type
"""
execute_snowflake_sql(query)

In [ ]:
# E. VIAJES POR HORA DEL DÍA Y DÍA DE SEMANA (PICOS)
print("ANÁLISIS E: Viajes por Hora del Día y Día de Semana (Picos)")

# Viajes por hora del día y día de semana para identificar picos
hourly_daily_patterns = (
    df.groupBy("pickup_hour", "day_of_week", "is_weekend")
    .agg(F.count("*").alias("total_trips"))
    .orderBy("pickup_hour", "day_of_week")
)

print("VIAJES POR HORA DEL DÍA Y DÍA DE SEMANA (PICOS):")
hourly_daily_patterns.show(50)

In [ ]:
# E. VIAJES POR HORA DEL DÍA Y DÍA DE SEMANA (PICOS)
print("ANÁLISIS E: Viajes por Hora del Día y Día de Semana (Picos)")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    pickup_hour,
    day_of_week,
    is_weekend,
    COUNT(*) AS total_trips
FROM analytics.obt_trips
WHERE pickup_hour IS NOT NULL
    AND day_of_week IS NOT NULL
    AND is_weekend IS NOT NULL
GROUP BY pickup_hour, day_of_week, is_weekend
ORDER BY pickup_hour, day_of_week;
"""
execute_snowflake_sql(query)

In [ ]:
# F. P50/P90 DE TRIP_DURATION_MIN POR BOROUGH DE PICKUP
print("ANÁLISIS F: P50/P90 de Trip_Duration_Min por Borough de Pickup")

# P50/P90 de trip_duration_min por borough de pickup
duration_percentiles = (
    df.groupBy("pu_borough")
    .agg(F.round(F.expr("percentile_approx(trip_duration_min, 0.5)"), 2).alias("p50_duration"),
         F.round(F.expr("percentile_approx(trip_duration_min, 0.9)"), 2).alias("p90_duration"))
    .orderBy("pu_borough")
)

print("P50/P90 DE TRIP_DURATION_MIN POR BOROUGH DE PICKUP:")
duration_percentiles.show()

In [ ]:
# F. P50/P90 DE TRIP_DURATION_MIN POR BOROUGH DE PICKUP
print("ANÁLISIS F: P50/P90 de Trip_Duration_Min por Borough de Pickup")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    pu_borough,
    APPROX_PERCENTILE(trip_duration_min, 0.5) AS p50_duration,
    APPROX_PERCENTILE(trip_duration_min, 0.9) AS p90_duration
FROM analytics.obt_trips
WHERE pu_borough IS NOT NULL
  AND trip_duration_min > 0
GROUP BY pu_borough
ORDER BY pu_borough;
"""
execute_snowflake_sql(query)

In [ ]:
# G. AVG_SPEED_MPH POR FRANJA HORARIA (6–9, 17–20) Y BOROUGH
print("ANÁLISIS G: Avg_Speed_Mph por Franja Horaria (6–9, 17–20) y Borough")

# Crear franja horaria y agrupar por franja y borough
speed_by_timeframe_borough = (
    df.withColumn("franja_horaria",
                  F.when((F.col("pickup_hour") >= 6) & (F.col("pickup_hour") <= 9), "Mañana (6-9h)")
                   .when((F.col("pickup_hour") >= 17) & (F.col("pickup_hour") <= 20), "Tarde (17-20h)")
                   .otherwise("Otras horas"))
    .filter(F.col("franja_horaria") != "Otras horas")
    .groupBy("franja_horaria", "pu_borough")
    .agg(F.round(F.avg("avg_speed_mph"), 2).alias("avg_speed_mph"))
    .orderBy("franja_horaria", "pu_borough")
)

print("AVG_SPEED_MPH POR FRANJA HORARIA (6–9, 17–20) Y BOROUGH:")
speed_by_timeframe_borough.show()

In [ ]:
# G. AVG_SPEED_MPH POR FRANJA HORARIA (6–9, 17–20) Y BOROUGH
print("ANÁLISIS G: Avg_Speed_Mph por Franja Horaria (6–9, 17–20) y Borough")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    CASE
        WHEN pickup_hour BETWEEN 6 AND 9 THEN 'Mañana (6-9h)'
        WHEN pickup_hour BETWEEN 17 AND 20 THEN 'Tarde (17-20h)'
    END AS franja_horaria,
    pu_borough,
    ROUND(AVG(avg_speed_mph), 2) AS avg_speed_mph
FROM analytics.obt_trips
WHERE pickup_hour IS NOT NULL
    AND pu_borough IS NOT NULL
    AND avg_speed_mph IS NOT NULL
    AND (
        (pickup_hour BETWEEN 6 AND 9) OR
        (pickup_hour BETWEEN 17 AND 20)
    )
GROUP BY
    CASE
        WHEN pickup_hour BETWEEN 6 AND 9 THEN 'Mañana (6-9h)'
        WHEN pickup_hour BETWEEN 17 AND 20 THEN 'Tarde (17-20h)'
    END,
    pu_borough
ORDER BY franja_horaria, pu_borough;
"""
execute_snowflake_sql(query)

In [ ]:
# H. PARTICIPACIÓN POR PAYMENT_TYPE_DESC Y SU RELACIÓN CON TIP_PCT
print("ANÁLISIS H: Participación por Payment_Type_Desc y su Relación con Tip_Pct")

# Participación por payment_type_desc y su relación con tip_pct
payment_participation_tips = (
    df.groupBy("payment_type_desc")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("tip_pct"), 2).alias("avg_tip_pct"),
         F.round(F.count("*") * 100.0 / df.count(), 2).alias("participacion_pct"))
    .orderBy(F.col("total_trips").desc())
)

print("PARTICIPACIÓN POR PAYMENT_TYPE_DESC Y SU RELACIÓN CON TIP_PCT:")
payment_participation_tips.show()

In [ ]:
# H. PARTICIPACIÓN POR PAYMENT_TYPE_DESC Y SU RELACIÓN CON TIP_PCT
print("ANÁLISIS H: Participación por Payment_Type_Desc y su Relación con Tip_Pct")

# Consulta SQL directa usando conector de Snowflake
query = """
WITH total_dataset AS (
    SELECT COUNT(*) as total_records
    FROM analytics.obt_trips
    WHERE payment_type_desc IS NOT NULL
)
SELECT
    payment_type_desc,
    COUNT(*) AS total_trips,
    ROUND(AVG(tip_pct), 2) AS avg_tip_pct,
    ROUND((COUNT(*) * 100.0 / (SELECT total_records FROM total_dataset)), 2) AS participacion_pct
FROM analytics.obt_trips
WHERE payment_type_desc IS NOT NULL
    AND tip_pct IS NOT NULL
GROUP BY payment_type_desc
ORDER BY COUNT(*) DESC;
"""
execute_snowflake_sql(query)

In [ ]:
# I. ¿QUÉ RATE_CODE_DESC CONCENTRAN MAYOR TRIP_DISTANCE Y TOTAL_AMOUNT?
print("ANÁLISIS I: ¿Qué Rate_Code_Desc concentran mayor Trip_Distance y Total_Amount?")

# Análisis de rate_code_desc por trip_distance y total_amount
rate_code_analysis = (
    df.groupBy("rate_code_desc")
    .agg(F.round(F.avg("trip_distance"), 2).alias("avg_trip_distance"),
         F.round(F.avg("total_amount"), 2).alias("avg_total_amount"),
         F.count("*").alias("total_trips"))
    .orderBy(F.col("avg_trip_distance").desc(), F.col("avg_total_amount").desc())
)

print("¿QUÉ RATE_CODE_DESC CONCENTRAN MAYOR TRIP_DISTANCE Y TOTAL_AMOUNT?:")
rate_code_analysis.show()

In [ ]:
# I. ¿QUÉ RATE_CODE_DESC CONCENTRAN MAYOR TRIP_DISTANCE Y TOTAL_AMOUNT?
print("ANÁLISIS I: ¿Qué Rate_Code_Desc concentran mayor Trip_Distance y Total_Amount?")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    rate_code_desc,
    ROUND(AVG(trip_distance), 2) AS avg_trip_distance,
    ROUND(AVG(total_amount), 2) AS avg_total_amount,
    COUNT(*) AS total_trips
FROM analytics.obt_trips
WHERE rate_code_desc IS NOT NULL
    AND trip_distance IS NOT NULL
    AND total_amount IS NOT NULL
GROUP BY rate_code_desc
ORDER BY AVG(trip_distance) DESC, AVG(total_amount) DESC;
"""
execute_snowflake_sql(query)

In [ ]:
# J. MIX YELLOW VS GREEN POR MES Y BOROUGH
print("ANÁLISIS J: Mix Yellow vs Green por Mes y Borough")

# Análisis del mix de servicios yellow vs green por mes y borough de pickup
yellow_green_mix_monthly = (
    df.filter(F.col("service_type").isin(["yellow", "green"]))
    .groupBy("year", "month", "pu_borough", "service_type")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("total_amount"), 2).alias("avg_fare"))
    .withColumn("total_by_month_borough", F.sum("total_trips").over(F.Window.partitionBy("year", "month", "pu_borough")))
    .withColumn("service_mix_pct", F.round((F.col("total_trips") * 100.0 / F.col("total_by_month_borough")), 2))
    .orderBy("year", "month", "pu_borough", "service_type")
)

print("MIX YELLOW VS GREEN POR MES Y BOROUGH:")
yellow_green_mix_monthly.show(50)

In [ ]:
# J. MIX YELLOW VS GREEN POR MES Y BOROUGH
print("ANÁLISIS J: Mix Yellow vs Green por Mes y Borough")

# Consulta SQL directa usando conector de Snowflake
query = """
WITH service_counts AS (
    SELECT
        year,
        month,
        pu_borough,
        service_type,
        COUNT(*) as total_trips,
        ROUND(AVG(total_amount), 2) as avg_fare
    FROM analytics.obt_trips
    WHERE service_type IN ('yellow', 'green')
        AND pu_borough IS NOT NULL
    GROUP BY year, month, pu_borough, service_type
),
monthly_totals AS (
    SELECT
        year,
        month,
        pu_borough,
        SUM(total_trips) as total_by_month_borough
    FROM service_counts
    GROUP BY year, month, pu_borough
)
SELECT
    sc.year,
    sc.month,
    sc.pu_borough,
    sc.service_type,
    sc.total_trips,
    sc.avg_fare,
    mt.total_by_month_borough,
    ROUND((sc.total_trips * 100.0 / mt.total_by_month_borough), 2) as service_mix_pct
FROM service_counts sc
JOIN monthly_totals mt
    ON sc.year = mt.year
    AND sc.month = mt.month
    AND sc.pu_borough = mt.pu_borough
ORDER BY sc.year, sc.month, sc.pu_borough, sc.service_type;
"""
execute_snowflake_sql(query)

In [ ]:
# K. TOP 20 FLUJOS PU→DO POR VOLUMEN Y SU TICKET PROMEDIO
print("ANÁLISIS K: Top 20 Flujos PU→DO por Volumen y su Ticket Promedio")

# Top 20 flujos pickup→dropoff por volumen y ticket promedio
top_flows_volume_ticket = (
    df.groupBy("pu_borough", "do_borough")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("total_amount"), 2).alias("avg_ticket"))
    .orderBy(F.col("total_trips").desc())
)

print("TOP 20 FLUJOS PU→DO POR VOLUMEN Y SU TICKET PROMEDIO:")
top_flows_volume_ticket.show(20)

In [ ]:
# K. TOP 20 FLUJOS PU→DO POR VOLUMEN Y SU TICKET PROMEDIO
print("ANÁLISIS K: Top 20 Flujos PU→DO por Volumen y su Ticket Promedio")


# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    pu_borough,
    do_borough,
    COUNT(*) AS total_trips,
    ROUND(AVG(total_amount), 2) AS avg_ticket
FROM analytics.obt_trips
WHERE pu_borough IS NOT NULL
    AND do_borough IS NOT NULL
    AND total_amount IS NOT NULL
GROUP BY pu_borough, do_borough
ORDER BY COUNT(*) DESC
LIMIT 20;
"""
execute_snowflake_sql(query)

In [ ]:
# L. DISTRIBUCIÓN DE PASSENGER_COUNT Y EFECTO EN TOTAL_AMOUNT
print("ANÁLISIS L: Distribución de Passenger_Count y Efecto en Total_Amount")

# Distribución de passenger_count y su efecto en total_amount
passenger_distribution_effect = (
    df.groupBy("passenger_count")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("total_amount"), 2).alias("avg_total_amount"),
         F.round(F.count("*") * 100.0 / df.count(), 2).alias("participacion_pct"))
    .orderBy("passenger_count")
)

print("DISTRIBUCIÓN DE PASSENGER_COUNT Y EFECTO EN TOTAL_AMOUNT:")
passenger_distribution_effect.show()

In [ ]:
# L. DISTRIBUCIÓN DE PASSENGER_COUNT Y EFECTO EN TOTAL_AMOUNT
print("ANÁLISIS L: Distribución de Passenger_Count y Efecto en Total_Amount")


# Consulta SQL directa usando conector de Snowflake
query = """
WITH total_dataset AS (
    SELECT COUNT(*) as total_records
    FROM analytics.obt_trips
    WHERE passenger_count IS NOT NULL
)
SELECT
    passenger_count,
    COUNT(*) AS total_trips,
    ROUND(AVG(total_amount), 2) AS avg_total_amount,
    ROUND((COUNT(*) * 100.0 / (SELECT total_records FROM total_dataset)), 2) AS participacion_pct
FROM analytics.obt_trips
WHERE passenger_count IS NOT NULL
    AND total_amount IS NOT NULL
GROUP BY passenger_count
ORDER BY passenger_count;
"""
execute_snowflake_sql(query)

In [ ]:
# M. IMPACTO DE TOLLS_AMOUNT Y CONGESTION_SURCHARGE POR ZONA
print("ANÁLISIS M: Impacto de Tolls_Amount y Congestion_Surcharge por Zona")

# Impacto de tolls_amount y congestion_surcharge por zona de pickup
tolls_congestion_impact_by_zone = (
    df.groupBy("pu_zone", "pu_borough")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("tolls_amount"), 2).alias("avg_tolls_amount"),
         F.round(F.avg("congestion_surcharge"), 2).alias("avg_congestion_surcharge"),
         F.round(F.avg("total_amount"), 2).alias("avg_total_amount"))
    .orderBy(F.col("avg_tolls_amount").desc(), F.col("avg_congestion_surcharge").desc())
)

print("IMPACTO DE TOLLS_AMOUNT Y CONGESTION_SURCHARGE POR ZONA:")
tolls_congestion_impact_by_zone.show(20)

In [ ]:
# M. IMPACTO DE TOLLS_AMOUNT Y CONGESTION_SURCHARGE POR ZONA
print("ANÁLISIS M: Impacto de Tolls_Amount y Congestion_Surcharge por Zona")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    pu_zone,
    pu_borough,
    COUNT(*) AS total_trips,
    ROUND(AVG(tolls_amount), 2) AS avg_tolls_amount,
    ROUND(AVG(congestion_surcharge), 2) AS avg_congestion_surcharge,
    ROUND(AVG(total_amount), 2) AS avg_total_amount
FROM analytics.obt_trips
WHERE pu_zone IS NOT NULL
    AND pu_borough IS NOT NULL
GROUP BY pu_zone, pu_borough
ORDER BY AVG(tolls_amount) DESC, AVG(congestion_surcharge) DESC
LIMIT 20;
"""
execute_snowflake_sql(query)

In [ ]:
# N. PROPORCIÓN DE VIAJES CORTOS VS LARGOS POR BOROUGH Y ESTACIONALIDAD
print("ANÁLISIS N: Proporción de Viajes Cortos vs Largos por Borough y Estacionalidad")

# Definir categorías de viaje (corto ≤ 3 millas, largo > 3 millas) y analizar por borough y estación
trip_length_seasonality = (
    df.withColumn("trip_category",
                  F.when(F.col("trip_distance") <= 3, "Corto (≤3 mi)")
                   .otherwise("Largo (>3 mi)"))
    .withColumn("season",
                F.when(F.col("month").isin([12, 1, 2]), "Invierno")
                 .when(F.col("month").isin([3, 4, 5]), "Primavera")
                 .when(F.col("month").isin([6, 7, 8]), "Verano")
                 .otherwise("Otoño"))
    .groupBy("pu_borough", "season", "trip_category")
    .agg(F.count("*").alias("total_trips"))
    .withColumn("total_by_borough_season", F.sum("total_trips").over(F.Window.partitionBy("pu_borough", "season")))
    .withColumn("proportion_pct", F.round((F.col("total_trips") * 100.0 / F.col("total_by_borough_season")), 2))
    .orderBy("pu_borough", "season", "trip_category")
)

print("PROPORCIÓN DE VIAJES CORTOS VS LARGOS POR BOROUGH Y ESTACIONALIDAD:")
trip_length_seasonality.show(50)

In [ ]:
# N. PROPORCIÓN DE VIAJES CORTOS VS LARGOS POR BOROUGH Y ESTACIONALIDAD
print("ANÁLISIS N: Proporción de Viajes Cortos vs Largos por Borough y Estacionalidad")

# Consulta SQL directa usando conector de Snowflake
query = """
WITH trips_enriched AS (
    SELECT
        pu_borough,
        CASE
            WHEN month IN (12, 1, 2) THEN 'Invierno'
            WHEN month IN (3, 4, 5) THEN 'Primavera'
            WHEN month IN (6, 7, 8) THEN 'Verano'
            ELSE 'Otoño'
        END AS season,
        CASE
            WHEN trip_distance <= 3 THEN 'Corto (≤3 mi)'
            ELSE 'Largo (>3 mi)'
        END AS trip_category
    FROM analytics.obt_trips
    WHERE pu_borough IS NOT NULL
      AND trip_distance IS NOT NULL
)

SELECT
    pu_borough,
    season,
    trip_category,
    COUNT(*) AS total_trips,
    ROUND(
        COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY pu_borough, season),
        2
    ) AS proportion_pct
FROM trips_enriched
GROUP BY pu_borough, season, trip_category
ORDER BY pu_borough, season, trip_category;

"""
execute_snowflake_sql(query)

In [ ]:
# O. DIFERENCIAS POR VENDOR EN AVG_SPEED_MPH Y TRIP_DURATION_MIN
print("ANÁLISIS O: Diferencias por Vendor en Avg_Speed_Mph y Trip_Duration_Min")

# Análisis de diferencias por vendor en velocidad promedio y duración de viaje
vendor_performance_comparison = (
    df.groupBy("vendor_name")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("avg_speed_mph"), 2).alias("avg_speed_mph"),
         F.round(F.avg("trip_duration_min"), 2).alias("avg_duration_min"),
         F.round(F.avg("trip_distance"), 2).alias("avg_distance"),
         F.round(F.avg("total_amount"), 2).alias("avg_total_amount"))
    .orderBy(F.col("avg_speed_mph").desc())
)

print("DIFERENCIAS POR VENDOR EN AVG_SPEED_MPH Y TRIP_DURATION_MIN:")
vendor_performance_comparison.show()

In [ ]:
# O. DIFERENCIAS POR VENDOR EN AVG_SPEED_MPH Y TRIP_DURATION_MIN
print("ANÁLISIS O: Diferencias por Vendor en Avg_Speed_Mph y Trip_Duration_Min")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    vendor_name,
    COUNT(*) AS total_trips,
    ROUND(AVG(avg_speed_mph), 2) AS avg_speed_mph,
    ROUND(AVG(trip_duration_min), 2) AS avg_duration_min,
    ROUND(AVG(trip_distance), 2) AS avg_distance,
    ROUND(AVG(total_amount), 2) AS avg_total_amount
FROM analytics.obt_trips
WHERE vendor_name IS NOT NULL
GROUP BY vendor_name
ORDER BY avg_speed_mph DESC;
"""
execute_snowflake_sql(query)

In [ ]:
# P. RELACIÓN MÉTODO DE PAGO ↔ TIP_AMOUNT POR HORA
print("ANÁLISIS P: Relación Método de Pago ↔ Tip_Amount por Hora")

# Análisis de la relación entre método de pago y cantidad de propina por hora del día
payment_tip_hourly_relationship = (
    df.groupBy("pickup_hour", "payment_type_desc")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("tip_amount"), 2).alias("avg_tip_amount"),
         F.round(F.avg("tip_pct"), 2).alias("avg_tip_pct"),
         F.round(F.avg("total_amount"), 2).alias("avg_total_amount"))
    .orderBy("pickup_hour", F.col("avg_tip_amount").desc())
)

print("RELACIÓN MÉTODO DE PAGO ↔ TIP_AMOUNT POR HORA:")
payment_tip_hourly_relationship.show(50)

In [ ]:
# P. RELACIÓN MÉTODO DE PAGO ↔ TIP_AMOUNT POR HORA
print("ANÁLISIS P: Relación Método de Pago ↔ Tip_Amount por Hora")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    pickup_hour,
    payment_type_desc,
    COUNT(*) AS total_trips,
    ROUND(AVG(tip_amount), 2) AS avg_tip_amount,
    ROUND(AVG(tip_pct), 2) AS avg_tip_pct,
    ROUND(AVG(total_amount), 2) AS avg_total_amount
FROM analytics.obt_trips
WHERE pickup_hour IS NOT NULL
    AND payment_type_desc IS NOT NULL
GROUP BY pickup_hour, payment_type_desc
ORDER BY pickup_hour, AVG(tip_amount) DESC
LIMIT 50;
"""
execute_snowflake_sql(query)

In [ ]:
# Q. ZONAS CON PERCENTIL 99 DE DURACIÓN/DISTANCIA FUERA DE RANGO (POSIBLE CONGESTIÓN/EVENTOS)
print("ANÁLISIS Q: Zonas con Percentil 99 de Duración/Distancia fuera de Rango (Posible Congestión/Eventos)")

# Identificar zonas con P99 extremos que podrían indicar congestión severa o eventos especiales
zones_p99_outliers = (
    df.groupBy("pu_zone", "pu_borough")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.expr("percentile_approx(trip_duration_min, 0.99)"), 2).alias("p99_duration"),
         F.round(F.expr("percentile_approx(trip_distance, 0.99)"), 2).alias("p99_distance"),
         F.round(F.avg("trip_duration_min"), 2).alias("avg_duration"),
         F.round(F.avg("trip_distance"), 2).alias("avg_distance"),
         F.round(F.avg("avg_speed_mph"), 2).alias("avg_speed"))
    .withColumn("duration_outlier_ratio", F.round(F.col("p99_duration") / F.col("avg_duration"), 2))
    .withColumn("distance_outlier_ratio", F.round(F.col("p99_distance") / F.col("avg_distance"), 2))
    .filter((F.col("duration_outlier_ratio") > 5) | (F.col("distance_outlier_ratio") > 4) | (F.col("p99_duration") > 120))
    .orderBy(F.col("duration_outlier_ratio").desc(), F.col("distance_outlier_ratio").desc())
)

print("ZONAS CON PERCENTIL 99 DE DURACIÓN/DISTANCIA FUERA DE RANGO:")
zones_p99_outliers.show(20)

In [ ]:
# Q. ZONAS CON PERCENTIL 99 DE DURACIÓN/DISTANCIA FUERA DE RANGO (POSIBLE CONGESTIÓN/EVENTOS)
print("ANÁLISIS Q: Zonas con Percentil 99 de Duración/Distancia fuera de Rango (Posible Congestión/Eventos)")

# Consulta SQL directa usando conector de Snowflake
query = """
WITH zone_metrics AS (
    SELECT
        pu_zone,
        pu_borough,
        COUNT(*) AS total_trips,
        APPROX_PERCENTILE(trip_duration_min, 0.99) AS p99_duration,
        APPROX_PERCENTILE(trip_distance, 0.99) AS p99_distance,
        ROUND(AVG(trip_duration_min), 2) AS avg_duration,
        ROUND(AVG(trip_distance), 2) AS avg_distance,
        ROUND(AVG(avg_speed_mph), 2) AS avg_speed
    FROM analytics.obt_trips
    WHERE pu_zone IS NOT NULL
      AND pu_borough IS NOT NULL
      AND trip_duration_min > 0
      AND trip_distance > 0
    GROUP BY pu_zone, pu_borough
)
SELECT
    pu_zone,
    pu_borough,
    total_trips,
    ROUND(p99_duration, 2) AS p99_duration,
    ROUND(p99_distance, 2) AS p99_distance,
    avg_duration,
    avg_distance,
    avg_speed,
    ROUND(p99_duration / NULLIF(avg_duration, 0), 2) AS duration_outlier_ratio,
    ROUND(p99_distance / NULLIF(avg_distance, 0), 2) AS distance_outlier_ratio
FROM zone_metrics
WHERE (p99_duration / NULLIF(avg_duration, 0) > 5)
   OR (p99_distance / NULLIF(avg_distance, 0) > 4)
   OR (p99_duration > 120)
ORDER BY duration_outlier_ratio DESC, distance_outlier_ratio DESC
LIMIT 20;
"""
execute_snowflake_sql(query)

In [ ]:
# R. YIELD POR MILLA (TOTAL_AMOUNT/TRIP_DISTANCE) POR BOROUGH Y HORA
print("ANÁLISIS R: Yield por Milla (Total_Amount/Trip_Distance) por Borough y Hora")

# Análisis del yield por milla (rentabilidad por distancia) por borough y hora del día
yield_per_mile_by_borough_hour = (
    df.filter(F.col("trip_distance") > 0)  # Evitar divisiones por cero
    .groupBy("pu_borough", "pickup_hour")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg(F.col("total_amount") / F.col("trip_distance")), 2).alias("yield_per_mile"),
         F.round(F.avg("total_amount"), 2).alias("avg_total_amount"),
         F.round(F.avg("trip_distance"), 2).alias("avg_trip_distance"),
         F.round(F.avg("tip_pct"), 2).alias("avg_tip_pct"))
    .orderBy("pu_borough", F.col("yield_per_mile").desc())
)

print("YIELD POR MILLA (TOTAL_AMOUNT/TRIP_DISTANCE) POR BOROUGH Y HORA:")
yield_per_mile_by_borough_hour.show(50)

In [ ]:
# R. YIELD POR MILLA (TOTAL_AMOUNT/TRIP_DISTANCE) POR BOROUGH Y HORA
print("ANÁLISIS R: Yield por Milla (Total_Amount/Trip_Distance) por Borough y Hora")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    pu_borough,
    pickup_hour,
    COUNT(*) AS total_trips,
    ROUND(AVG(total_amount / trip_distance), 2) AS yield_per_mile,
    ROUND(AVG(total_amount), 2) AS avg_total_amount,
    ROUND(AVG(trip_distance), 2) AS avg_trip_distance,
    ROUND(AVG(tip_pct), 2) AS avg_tip_pct
FROM analytics.obt_trips
WHERE trip_distance > 0
    AND pu_borough IS NOT NULL
    AND pickup_hour IS NOT NULL
GROUP BY pu_borough, pickup_hour
ORDER BY pu_borough, AVG(total_amount / trip_distance) DESC
LIMIT 50;
"""
execute_snowflake_sql(query)

In [ ]:
# S. CAMBIOS YOY EN VOLUMEN Y TICKET PROMEDIO POR SERVICE_TYPE
print("ANÁLISIS S: Cambios YoY en Volumen y Ticket Promedio por Service_Type")

# Análisis de cambios año a año en volumen de viajes y ticket promedio por tipo de servicio
yoy_volume_ticket_changes = (
    df.groupBy("year", "service_type")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("total_amount"), 2).alias("avg_ticket"),
         F.round(F.sum("total_amount"), 2).alias("total_revenue"))
    .withColumn("prev_year_trips", F.lag("total_trips").over(F.Window.partitionBy("service_type").orderBy("year")))
    .withColumn("prev_year_ticket", F.lag("avg_ticket").over(F.Window.partitionBy("service_type").orderBy("year")))
    .withColumn("volume_change_pct", F.round(((F.col("total_trips") - F.col("prev_year_trips")) / F.col("prev_year_trips")) * 100, 2))
    .withColumn("ticket_change_pct", F.round(((F.col("avg_ticket") - F.col("prev_year_ticket")) / F.col("prev_year_ticket")) * 100, 2))
    .orderBy("service_type", "year")
)

print("CAMBIOS YOY EN VOLUMEN Y TICKET PROMEDIO POR SERVICE_TYPE:")
yoy_volume_ticket_changes.show(50)

In [ ]:
# S. CAMBIOS YOY EN VOLUMEN Y TICKET PROMEDIO POR SERVICE_TYPE
print("ANÁLISIS S: Cambios YoY en Volumen y Ticket Promedio por Service_Type")

# Consulta SQL directa usando conector de Snowflake
query = """
WITH yearly_metrics AS (
    SELECT
        year,
        service_type,
        COUNT(*) as total_trips,
        ROUND(AVG(total_amount), 2) as avg_ticket,
        ROUND(SUM(total_amount), 2) as total_revenue
    FROM analytics.obt_trips
    WHERE service_type IS NOT NULL
    GROUP BY year, service_type
)
SELECT
    year,
    service_type,
    total_trips,
    avg_ticket,
    total_revenue,
    LAG(total_trips) OVER (PARTITION BY service_type ORDER BY year) as prev_year_trips,
    LAG(avg_ticket) OVER (PARTITION BY service_type ORDER BY year) as prev_year_ticket,
    ROUND(((total_trips - LAG(total_trips) OVER (PARTITION BY service_type ORDER BY year)) * 100.0 /
           LAG(total_trips) OVER (PARTITION BY service_type ORDER BY year)), 2) as volume_change_pct,
    ROUND(((avg_ticket - LAG(avg_ticket) OVER (PARTITION BY service_type ORDER BY year)) * 100.0 /
           LAG(avg_ticket) OVER (PARTITION BY service_type ORDER BY year)), 2) as ticket_change_pct
FROM yearly_metrics
ORDER BY service_type, year
LIMIT 50;
"""
execute_snowflake_sql(query)

In [ ]:
# T. DÍAS CON ALTA CONGESTION_SURCHARGE - EFECTO EN TOTAL_AMOUNT VS DÍAS NORMALES
print("ANÁLISIS T: Días con Alta Congestion_Surcharge - Efecto en Total_Amount vs Días Normales")

# Análisis del efecto de la congestión en el total_amount comparando días con alta congestión vs días normales
congestion_effect_analysis = (
    df.withColumn("has_congestion", F.when(F.col("congestion_surcharge") > 0, "Con Congestión").otherwise("Sin Congestión"))
    .groupBy("pickup_date", "has_congestion")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("total_amount"), 2).alias("avg_total_amount"),
         F.round(F.avg("congestion_surcharge"), 2).alias("avg_congestion_surcharge"),
         F.round(F.avg("trip_duration_min"), 2).alias("avg_duration"),
         F.round(F.avg("avg_speed_mph"), 2).alias("avg_speed"))
    .withColumn("daily_trips", F.sum("total_trips").over(F.Window.partitionBy("pickup_date")))
    .withColumn("congestion_pct", F.round((F.col("total_trips") * 100.0 / F.col("daily_trips")), 2))
    .filter(F.col("has_congestion") == "Con Congestión")
    .withColumn("high_congestion_day", F.when(F.col("congestion_pct") >= 30, "Día Alta Congestión").otherwise("Día Normal"))
    .orderBy("pickup_date")
)

print("DÍAS CON ALTA CONGESTION_SURCHARGE - EFECTO EN TOTAL_AMOUNT VS DÍAS NORMALES:")
congestion_effect_analysis.show(50)

In [ ]:
# T. DÍAS CON ALTA CONGESTION_SURCHARGE - EFECTO EN TOTAL_AMOUNT VS DÍAS NORMALES
print("ANÁLISIS T: Días con Alta Congestion_Surcharge - Efecto en Total_Amount vs Días Normales")

query = """
WITH daily_stats AS (
    SELECT
        pickup_date,
        CASE WHEN congestion_surcharge > 0 THEN 'Con Congestión' ELSE 'Sin Congestión' END AS has_congestion,
        COUNT(*) AS total_trips,
        ROUND(AVG(total_amount), 2) AS avg_total_amount,
        ROUND(AVG(congestion_surcharge), 2) AS avg_congestion_surcharge,
        ROUND(AVG(trip_duration_min), 2) AS avg_duration,
        ROUND(AVG(avg_speed_mph), 2) AS avg_speed
    FROM analytics.obt_trips
    WHERE pickup_date IS NOT NULL
    GROUP BY pickup_date, has_congestion
),
with_daily_totals AS (
    SELECT
        *,
        SUM(total_trips) OVER (PARTITION BY pickup_date) AS daily_trips,
        ROUND((total_trips * 100.0 / SUM(total_trips) OVER (PARTITION BY pickup_date)), 2) AS congestion_pct
    FROM daily_stats
)
SELECT
    pickup_date,
    has_congestion,
    total_trips,
    avg_total_amount,
    avg_congestion_surcharge,
    avg_duration,
    avg_speed,
    daily_trips,
    congestion_pct,
    CASE WHEN congestion_pct >= 30 THEN 'Día Alta Congestión' ELSE 'Día Normal' END AS high_congestion_day
FROM with_daily_totals
WHERE has_congestion = 'Con Congestión'
ORDER BY pickup_date
LIMIT 50;
"""
execute_snowflake_sql(query)